#### Задание 1
Импортируйте библиотеки pandas и numpy.<br>
Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn..<br>
Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент random state должен быть равен 42.<br>
Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.<br>
Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
boston = load_boston()
data = boston.data
target = boston.target
feature_names = boston.feature_names

In [3]:
x = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=['price'])

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [5]:
lr = LinearRegression()

In [6]:
lr.fit(x_train, y_train)

LinearRegression()

In [7]:
y_pred = lr.predict(x_test)

In [8]:
check_test = pd.DataFrame({'y_test': y_test['price'], 'y_pred': y_pred.flatten()}, columns=['y_test', 'y_pred'])

In [9]:
check_test.head()

,y_test,y_pred
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280


#### Задание 2
Создайте модель под названием model с помощью класса RandomForestRegressor из модуля sklearn.ensemble. Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42.<br>
Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values\[:, 0\], чтобы получить из датафрейма одномерный массив Numpy, так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.<br>
Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.<br>
Напишите в комментариях к коду, какая модель в данном случае работает лучше.<br>


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [11]:
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

In [12]:
model.fit(x_train, y_train.values[:,0])

RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)

In [13]:
y_pred2 = model.predict(x_test)

In [14]:
check_test2 = pd.DataFrame(
    {'y_test': y_test['price'], 'y_pred2': y_pred2.flatten()}, columns=['y_test', 'y_pred2'])

In [15]:
r2_score(y_test, y_pred)

0.7112260057484973

In [16]:
r2_score(y_test, y_pred2)

0.87472606157312

In [17]:
check_test.merge(check_test2)

,y_test,y_pred,y_pred2
0,23.6,28.648960,22.806412
1,32.4,36.495014,31.131464
2,13.6,15.411193,16.339125
3,22.8,25.403213,23.810726
4,16.1,18.855280,17.139521
...,...,...,...
243,19.3,16.736336,18.473664
244,8.5,6.128584,6.912249
245,17.0,23.340072,16.941018
246,14.3,16.682863,15.586103


На основе сравнения предсказаний и оценки метрики r2 можно заключить, что модель model, использующая RandomForestRegressor работает лучше чем модель lr, используящая LinearRegression

#### *Задание 3
Вызовите документацию для класса RandomForestRegressor,
найдите информацию об атрибуте feature_importances_.<br>
С помощью этого атрибута найдите сумму всех показателей важности,
установите, какие два признака показывают наибольшую важность.


In [18]:
help(RandomForestRegressor.feature_importances_)

Help on property:

    The impurity-based feature importances.
    
    The higher, the more important the feature.
    The importance of a feature is computed as the (normalized)
    total reduction of the criterion brought by that feature.  It is also
    known as the Gini importance.
    
    high cardinality features (many unique values). See
    :func:`sklearn.inspection.permutation_importance` as an alternative.
    
    Returns
    -------
    feature_importances_ : ndarray of shape (n_features,)
        The values of this array sum to 1, unless all trees are single node
        trees consisting of only the root node, in which case it will be an
        array of zeros.



In [19]:
sum(model.feature_importances_)

1.0

In [20]:
higher_important = sorted(model.feature_importances_)[-2:]
higher_important

[0.4026817857034993, 0.4158473181914483]

In [21]:
for n, i in enumerate(model.feature_importances_):
    if i in higher_important:
        print(x_train.columns[n], end=' ')

RM LSTAT 

#### *Задание 4
В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection.Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакции по кредитной карте являются мошенническими.Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки),так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель.Мы будем вычислять AUC, то есть площадь под кривой ROC.<br>
Импортируйте из соответствующих модулей RandomForestClassifier, GridSearchCV и train_test_split.<br>
Загрузите датасет creditcard.csv и создайте датафрейм df.<br>
С помощью метода value_counts с аргументом normalize=True убедитесь в том, что выборка несбалансирована. Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков. Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма:<br>
pd.options.display.max_columns = 100.<br>
Просмотрите первые 10 строк датафрейма df.<br>
Создайте датафрейм X из датафрейма df, исключив столбец Class.<br>
Создайте объект Series под названием y из столбца Class.<br>
Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции train_test_split, используя аргументы: test_size=0.3, random_state=100, stratify=y.<br>
У вас должны получиться объекты X_train, X_test, y_train и y_test.<br>
Просмотрите информацию о их форме.<br>
Для поиска по сетке параметров задайте такие параметры:<br>
parameters = \[{'n_estimators': [10, 15],
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}\]<br>
Создайте модель GridSearchCV со следующими аргументами:<br>
estimator=RandomForestClassifier(random_state=100),<br>
param_grid=parameters,<br>
scoring='roc_auc',<br>
cv=3.<br>
Обучите модель на тренировочном наборе данных (может занять несколько минут).<br>
Просмотрите параметры лучшей модели с помощью атрибута best_params_.<br>
Предскажите вероятности классов с помощью полученной модели и метода predict_proba.<br>
Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba. Из модуля sklearn.metrics импортируйте метрику roc_auc_score.<br>
Вычислите AUC на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументов массивы y_test и y_pred_proba.<br>


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [23]:
df = pd.read_csv('creditcard.csv')

In [24]:
df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [26]:
pd.options.display.max_columns = 100

In [27]:
X = df.drop(['Class'], axis=1)

In [28]:
y = df['Class']

In [29]:
type(y)

pandas.core.series.Series

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [31]:
X_train.shape

(199364, 30)

In [32]:
y_train.shape

(199364,)

In [33]:
parameters = [
    {'n_estimators': [10, 15],
     'max_features': np.arange(3, 5),
     'max_depth': np.arange(4, 7)}]

In [34]:
model = GridSearchCV(estimator=RandomForestClassifier(random_state=100),
                   param_grid=parameters,
                   scoring='roc_auc',
                   cv=3)

In [35]:
model.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=100),
             param_grid=[{'max_depth': array([4, 5, 6]),
                          'max_features': array([3, 4]),
                          'n_estimators': [10, 15]}],
             scoring='roc_auc')

In [36]:
model.best_params_

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

In [37]:
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [38]:
roc_auc_score(y_test, y_pred_proba)

0.9462664156037156